# `ctypes` Tips #

[`ctypes`](https://docs.python.org/3/library/ctypes.html) is a very handy tool for building Python wrappers for shared libraries written for C or C++. In most cases, it is probably preferable to use this, rather than write an *extension module* in C or C++ to provide the Python API: it can take a lot of code to implement the necessary C/C++ wrappers to represent Python objects and methods, while this can usually be done directly in Python with a fraction of the effort.

While the documentation for `ctypes` is quite comprehensive, there are a few subtle points that might not be clear.

A Python wrapper will typically need a lot of things from the `ctypes` module. Its own documentation page uses wildcard imports in the examples, which I prefer to avoid. Instead, I reference its exports by importing the module under a shorter name:

In [ ]:
import ctypes as ct

## Load The Runtime Library, Not The Development Library ##

Consider the following directory entries currently on my Debian system for the [Cairo](https://cairographics.org/) graphics library:

    /usr/lib/x86_64-linux-gnu/libcairo.so -> libcairo.so.2.11600.0
    /usr/lib/x86_64-linux-gnu/libcairo.so.2 -> libcairo.so.2.11600.0
    /usr/lib/x86_64-linux-gnu/libcairo.so.2.11600.0

As you can see, there are 3 separate names for the same file. Which one should you use?

The answer is, use the name `libcairo.so.2`. The unversioned name comes from the *development* package:

    > dpkg-query -S /usr/lib/x86_64-linux-gnu/libcairo.so
    libcairo2-dev:amd64: /usr/lib/x86_64-linux-gnu/libcairo.so

while the versioned names come from the *runtime* package:

    > dpkg-query -S /usr/lib/x86_64-linux-gnu/libcairo.so.2
    libcairo2:amd64: /usr/lib/x86_64-linux-gnu/libcairo.so.2

So, in a wrapper for Cairo, you would load the library using something like

    cairo = ct.cdll.LoadLibrary("libcairo.so.2")

You only need to care about the first numeric component of the version, since that is the one incremented for any ABI changes (which might necessitate changes to your wrapper).

While having the development package installed is useful while you are developing your wrapper (being able to refer to the include files for information, etc), you should only require your users to have the runtime package in order to be able to run scripts that use your wrapper. Of course, they, too might find the development package useful when writing such scripts. But let that be their choice.

This only applies to distros like Debian which publish their packages in precompiled binary form. In ones like Gentoo, where users install everything from source, there is no distinction between “development” and “runtime” packages.

## `c_void_p` ##

The `ctypes` explanation of `c_void_p` (the untyped pointer) is that the Python type is `int` or `None`.  When creating a `c_void_p`, you can pass an integer for the address (including 0 for `NULL`), or you can pass `None` as an alternative for `NULL`. But when getting back one of these, the 0 or `NULL` address is always converted to `None`:

In [ ]:
p1 = ct.c_void_p(3)
p2 = ct.c_void_p(0)
print(p1.value, p2.value)

Note that, while other pointer types have a `contents` attribute you can use to dereference the pointer, `c_void_p` does not.

## Getting Addresses Of Python Objects ##

Sometimes you want to pass the address of the data inside a Python object directly to a library routine, to save copying data back and forth. This is particularly useful for Python objects of type `bytes` and `bytearray`, as well as arrays created with the [`array`](https://docs.python.org/3/library/array.html) module. This has to be done in slightly different ways for these different objects.

To demonstrate this, I will make calls to the low-level `libc` [`memcpy`(3)](https://linux.die.net/man/3/memcpy) routine to copy data between Python objects:

In [ ]:
libc = ct.cdll.LoadLibrary("libc.so.6")
libc.memcpy.restype = ct.c_void_p
libc.memcpy.argtypes = (ct.c_void_p, ct.c_void_p, ct.c_size_t) # dst, src, count

For a `bytes` object, a simple `cast` is sufficient to obtain the address of the data:

In [ ]:
b1 = b"some:text"
b2 = b"other text"
print(b1, b2)
b1adr = ct.cast(b1, ct.c_void_p).value
b2adr = ct.cast(b2, ct.c_void_p).value
libc.memcpy(b2adr, b1adr, 5)
print(b1, b2)

For a `bytearray`, things are slightly more involved.

In [ ]:
b1 = bytearray(b"different text")
b1adr = ct.addressof((ct.c_ubyte * len(b1)).from_buffer(b1))
libc.memcpy(b2adr, b1adr, 6)
print(b1, b2)

By the way, you can’t use this technique on `bytes`; it appears this only works on *mutable* objects.

[`array`](https://docs.python.org/3/library/array.html) arrays have a `buffer_info()` method which returns the address and length of the underlying memory buffer. While this still works, it is apparently deprecated. So the same trick works as for `bytearray`s:

In [ ]:
import array
b1 = array.array("B", b"yet other text")
b1adr = ct.addressof((ct.c_ubyte * len(b1)).from_buffer(b1))
libc.memcpy(b2adr, b1adr, 7)
print(b1.tobytes(), b2)

Casting can be used to create a pointer to a `ctypes` array type.

In [ ]:
b = bytearray(b"some text")
b1 = (ct.c_ubyte * 0).from_buffer(b)

In this case, I have set the array length to 0, which prevents me from using `b1` directly to access any of the bytes in `b`, but a pointer constructed from `b1` is not so constrained:

In [ ]:
p = ct.cast(b1, ct.POINTER(ct.c_ubyte))
[chr(c) for c in p[0:3]]

Because the original Python object is mutable, `ctypes` allows me to use the pointer to assign to its components from within Python (this would not be allowed for a pointer into a `bytes` object, for example):

In [ ]:
p[5] = ord("z")
b

Of course, external libraries are not going to respect Python’s access-control mechanisms.

## `c_char` And `c_char_p` ##

A `c_char_p` is not quite equivalent to `ct.POINTER(c_char)`; it is assumed to point to a *null-terminated* array of `c_char`. Accessing the `value` attribute returns the data up to, but not including, the terminating null:

In [ ]:
b = b"hello\0 there"
ct.cast(b, ct.c_char_p).value

Note you cannot assign to the `value` or `contents` of a `c_char_p` (this silently reallocates the buffer to hold the new value):

In [ ]:
ct.cast(b, ct.c_char_p).contents = b"text"
b

But you can to the `value` of an _array_ of `c_char` (note the extra null inserted after the value):

In [ ]:
ct.cast(b, ct.POINTER(len(b) * ct.c_char))[0][0:4] = (4 * ct.c_char)(*list(b"text"))
b

Here’s a similar thing done to a `bytearray` instead of a `bytes` object:

In [ ]:
b = bytearray(b"hello\0 there")
(len(b) * ct.c_char).from_buffer(b).value = b"tex"
b

## Pointers To Simple Types ##

Dereferencing a pointer to a simple type can be done either via the `contents` attribute or by array indexing. But note that `contents` returns a reference to the `ctypes` object holding the value; this in turn has a `value` attribute that you can use to change the value.

In [ ]:
i1 = ct.c_int(3)
i2 = ct.c_int(3)
p1 = ct.pointer(i1)
p2 = ct.pointer(i2)
print(p1.contents, p2[0])
p1.contents.value = 2 # “p1.contents = 2” won’t work
p2[0] = 5
print(p1.contents, p2.contents)


## Example: Wrapper For Cairo ##

In order to offer specific examples, the following discussion will refer to details of the implementation of **Qahirah**, my Python wrapper for the [**Cairo**](http://cairographics.org/) graphics library. This can be obtained from [GitLab](https://gitlab.com/ldo/qahirah) or [GitHub](https://github.com/ldo/qahirah).

## Reference-Counted Objects ##

Some libraries have their own systems for keeping track of reference counts of objects. It’s probably easier to avoid library objects having a reference count greater than 1; just let Python itself keep track of references to your Python wrapper object, and have your `__del__()` method dispose of the library object. Then the library object automatically exists for as long as your Python wrapper object exists.

## Round-Trip Object Mapping ##

The underlying library may implement several different types of objects, and objects of one type may hold references to objects of another type. There will likely be API calls not only to set such references to particular objects, but also to retrieve the current object references from the referencing objects. How do you map this back to Python?

For example, in Cairo, a drawing `Context` can have a current _source_ for computing rendered pixel values, which is a reference to a `Pattern`. You have a `set_source` call to set the source to a specified `Pattern`, and you have a `get_source` call to retrieve a reference to the last-set `Pattern`.

It is easy enough to wrap the `set_source` call in an equivalent Python method on the `Context` wrapper, which takes a `Pattern` wrapper and extracts and passes both underlying objects to the library call. But what about `get_source`? Ideally, if you already have a Python wrapper object for that `Pattern`, it would be good to return that, rather than create a new wrapper.

One way to do this would be to have a separate field in your `Context` wrapper which saves the last-set `Pattern` wrapper when the `set_source` method is called, then your `get_source` method can “cheat” and, rather than calling the Cairo routine at all, it simply returns the value of this field. This has some consequences I find undesirable:
* it means the wrapper object has to be kept around, even if the user doesn’t need it.
* it stands the risk of getting out of sync, if the user makes a direct call to the Cairo `set_source` routine via some other way.

There is a more elegant solution, where you keep a class variable in the referenced class (`Pattern`, in this case) which holds a mapping from underlying object references to the corresponding Python objects; then the `get_source` method can simply look up the returned value in this dictionary; if an entry already exists, it can return that existing wrapper, otherwise it can automatically construct a new wrapper, add it to the dictionary, and return it.

A further twist is to make this dictionary a `WeakValueDictionary`. That way, entries in it only exist for as long as the caller has other references to those Python wrapper objects; if the user gets rid of their own references, then they disappear from this mapping as well. This keeps memory usage for unwanted objects from growing without bounds.

In order to make this work, the constructor for the wrapper object has to be implemented entirely within the `__new__()` method, rather than using `__init__()`. This way, you can control whether a new wrapper object is created or not.


## Avoiding Segfaults On Program Exit ##

There is a drawback with custom `__del__()` methods on Python objects, which surfaces at program exit time: **there are no guarantees about the order in which they are called at exit time**. For example, if the underlying library object has already been deleted, then attempts to call disposal routines can cause intermittent segfaults. Of course, at times other than program termination, `__del__()` cleanup should continue to work OK.

The answer is for your wrapper module to install an `atexit` cleanup routine which **deletes the `__del__()` methods from all your classes that have them**. At the time that `atexit` routines are called, the state of things is still guaranteed to be sane; they will only go to pieces afterwards. For example, in my Cairo wrapper above, among the last few lines of code is something like

    def _atexit() :
        for cls in Context, Surface, Device, Pattern, ... etc ... :
            delattr(cls, "__del__")
        #end for
    #end _atexit
    atexit.register(_atexit)

Now, any of these objects that are disposed after this will no longer try to call any `__del__()` methods, which should be OK because everything is going away anyway.